# Постановка задачи

Задачей предусматриваются перевозки от поставщиков к потребителям по прямым маршрутам. Обозначим множество(количество) поставщиков через Warhouses, множество(количество) потребителей – Consumers

Для каждого маршрута Warhouse-Consumer задана стоимость перевозки 
 〖routeCost〗_ij,i=1..Warhouses,j=1..Consumers.
 
На балансе каждого поставщика имеется объем товара, доступный к перевозке: 〖WareVolume〗_i,i=1..Warhouses.

Для каждого потребителя задан спрос (количество товара, которое необходимо привезти потребителю): 〖Consumers〗_j,j=1..Consumers.

Цель: исходя из спроса на потребителях и предложения на поставщиках, построить план перевозок от поставщиков к потребителям при минимизации издержек на транспортировку.

Задача со звездочкой: провести анализ на чувствительность (для компьютерного метода решения).

Задачу необходимо решить двумя способами: 

1)	Решить руками на бумаге любым способом (можно решать как с помощью известных методов, так и придумать собственный)

2)	Решить в Excel 2010 или на любом из следующих языков программирования: C++, C#, Java, Matlab, OPL

# Формулировка модели

Sets:

$W$ – warehouses

$C$ – customers

Parameters:

$S_{w}$ – supply at warehouse $w$

$D_{c}$ – demand by customer $c$

$U_{w,c}$ – unit transport cost from $w$ to $c$

Variables:

$x_{w,c}$ – transport value from $w$ to $c$

$z$ – total transport cost

Minimize: 

$z = \sum_{w, c} U_{w,c} · x_{w,c}$

subject to:

$\sum_{c}x_{w,c} ≤ S_{w}$

$\sum_{w}x_{w,c} ≥ D_{c}$

$x_{w,c} ≥ 0  ∀(w,c)$ 


# Реализация

Импортируем библиотеки

In [1]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

Берем данные из внешего источника

In [2]:
path = (r"01 Data.xls") # Относительный путь к файлу
df_demand = pd.read_excel(path, sheet_name='Demand') # Считываем датафрейм из файла
df_supply = pd.read_excel(path, sheet_name='Supply') # Считываем датафрейм из файла
df_costs = pd.read_excel(path, sheet_name='Costs') # Считываем датафрейм из файла

Начинаем писать модель

In [3]:
model = pyo.ConcreteModel('Transport problem')

Множества

In [4]:
model.customers = pyo.Set(initialize = list(df_demand.set_index(['cons_name']).index))
model.warehouses = pyo.Set(initialize = list(df_supply.set_index(['ware_name']).index))
model.costs = pyo.Set(initialize = list(df_costs.set_index(['ware_name', 'cons_name']).index)) # добавляем множество для пары индексов 

Параметры

In [5]:
model.demand = pyo.Param(model.customers, initialize = df_demand.set_index(['cons_name'])['cons_demand, кг товара'].to_dict())
model.supply = pyo.Param(model.warehouses, initialize = df_supply.set_index(['ware_name'])['ware_volume, кг товара'].to_dict())
model.cost = pyo.Param(model.costs, initialize = df_costs.set_index(['ware_name', 'cons_name'])['transport_cost (стоимость в у.е.)'].to_dict())

Переменные

In [6]:
model.x = pyo.Var(model.costs, domain=pyo.NonNegativeReals)

Целевая функция

In [7]:
def OF_rule(model):
    return sum(model.x[(w, c)] * model.cost[(w, c)] for (w, c) in model.costs)
model.c = pyo.Objective(rule=OF_rule, sense=pyo.minimize)

Ограничения

In [8]:
def Supply_rule(model, w):
    return sum(model.x[(w, c)] for c in model.customers) <= model.supply[w]
model.SupplyConstr = pyo.Constraint(model.warehouses, rule=Supply_rule)

In [9]:
def Demand_rule(model, c):
    return sum(model.x[(w, c)] for w in model.warehouses) >= model.demand[c] # Хотя тут обсуждаемо – по постановке может быть и строгое равенство, но т.к. спрос равен предложению, то без разницы
model.DemandConstr = pyo.Constraint(model.customers, rule=Demand_rule)

Выбираем солвер

In [10]:
solver = pyo.SolverFactory('gurobi')

Создаем суффикс для анализа чувствительсности

In [11]:
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Решаем модель

In [12]:
result = solver.solve(model)

Смотрим на целевую

In [13]:
model.c()

1070.0

Выводим результаты

In [14]:
print(result)


Problem: 
- Name: x1
  Lower bound: 1070.0
  Upper bound: 1070.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 24
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 24
  Number of nonzeros: 48
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.05900001525878906
  Error rc: 0
  Time: 0.4662325382232666
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Проверяем, как сгенерировалась модель

In [15]:
model.display()

Model Transport problem

  Variables:
    x : Size=24, Index=costs
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        ('War1', 'Cons1') :     0 :  10.0 :  None : False : False : NonNegativeReals
        ('War1', 'Cons2') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('War1', 'Cons3') :     0 :  40.0 :  None : False : False : NonNegativeReals
        ('War1', 'Cons4') :     0 :  20.0 :  None : False : False : NonNegativeReals
        ('War1', 'Cons5') :     0 :  10.0 :  None : False : False : NonNegativeReals
        ('War1', 'Cons6') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('War2', 'Cons1') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('War2', 'Cons2') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('War2', 'Cons3') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('War2', 'Cons4') :     0 :   0.0 :  None : False : False : NonNegativeReals
        

Записываем lp-файл с моделью

In [16]:
model.write('01 model.lp', io_options={'symbolic_solver_labels': True})

('01 model.lp', 1844881906512)

## Анализ на чувствительность 

In [17]:
print("Duals")
for c in model.component_objects(pyo.Constraint, active=True):
    print("   Constraint", c)
    for index in c:
        print("      ", index, model.dual[c[index]])

Duals
   Constraint SupplyConstr
       War1 0.0
       War2 -1.0
       War3 -5.0
       War4 -3.0
   Constraint DemandConstr
       Cons1 3.0
       Cons2 5.0
       Cons3 8.0
       Cons4 13.0
       Cons5 4.0
       Cons6 1.0


Изменение спроса на единицу увеличит целевую на величину значения двойственной оценки для DemandConstr